# Telegram bot

![](images/telegram.jpg)

Github проекта:  
https://github.com/python-telegram-bot/python-telegram-bot  
Документация:  
https://github.com/python-telegram-bot/python-telegram-bot/wiki  


```bash
pip install python-telegram-bot
```

Примеры можно посмотреть здесь:  
https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/

In [ ]:
!pip install python-telegram-bot

Погодите-ка, сперва надо завести самого бота!  
  
### Bot Father
  
В поиске Telegram находим Bot Farher'a и создаем своего бота с помощью команды `/newbot`. Затем вводим имя и юзернейм. Обратите внимание, что юзернейм должен оканчиваться на `bot`!


![](images/how-to.png)

Для нашего бота был сгенерирован уникальный непротухающий токен доступа к Телеграм:  
`1774364697:AAFLXREpzYbaFg0QZLETiNGEdpf5kH3uylc`  
**Если вздумаете создать своего бота, используйте иной токен!**

In [ ]:
import html
import json
import logging
import traceback

from telegram import ParseMode
from telegram import Update
from telegram.ext import CallbackContext
from telegram.ext import CommandHandler
from telegram.ext import Filters
from telegram.ext import MessageHandler
from telegram.ext import Updater


# top secret, not for distribution
BOT_TOKEN = "1774364697:AAFLXREpzYbaFg0QZLETiNGEdpf5kH3uylc"
DEVELOPER_CHAT_ID = "333950593"


# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments bot and
# update. Error handlers also receive the raised TelegramError object in error.    
def start(update: Update, _: CallbackContext):
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        f'Hi {user.mention_markdown_v2()}\!'
    )


def help_command(update: Update, _: CallbackContext):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext):
    """Echo the user message."""
    logger.info(f"Receiving message from @{update.message.chat.username}")
    update.message.reply_text(update.message.text)


def error_handler(update: Update, context: CallbackContext):
    """Log the error and send a telegram message to notify the developer."""
    # Log the error before we do anything else, so we can see it even if something breaks.
    logger.error(msg="Exception while handling an update:", exc_info=context.error)

    # traceback.format_exception returns the usual python message about an exception, but as a
    # list of strings rather than a single string, so we have to join them together.
    tb_list = traceback.format_exception(None, context.error, context.error.__traceback__)
    tb_string = ''.join(tb_list)

    # Build the message with some markup and additional information about what happened.
    # You might need to add some logic to deal with messages longer than the 4096 character limit.
    update_str = update.to_dict() if isinstance(update, Update) else str(update)
    message = (
        f'An exception was raised while handling an update\n'
        f'<pre>update = {html.escape(json.dumps(update_str, indent=2, ensure_ascii=False))}'
        '</pre>\n\n'
        f'<pre>context.chat_data = {html.escape(str(context.chat_data))}</pre>\n\n'
        f'<pre>context.user_data = {html.escape(str(context.user_data))}</pre>\n\n'
        f'<pre>{html.escape(tb_string)}</pre>'
    )

    # Finally, send the message
    context.bot.send_message(chat_id=DEVELOPER_CHAT_ID, text=message, parse_mode=ParseMode.HTML)


def launch_bot():
    """Start the bot."""
    # Create the EventHandler and pass it your bot's token.
    updater = Updater(BOT_TOKEN)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # log all errors
    dp.add_error_handler(error_handler)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
launch_bot()